In [1]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Twitter
import sentencepiece as spm
import re
import string
from bs4 import BeautifulSoup
import urllib.request
import feedparser
import requests
from collections import defaultdict
from operator import itemgetter
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd

In [2]:
import pickle
with open('dc_LOL_newWord.pkl', 'rb') as f:
    words = pickle.load(f)

In [3]:
with open('DC_LOL_preprocessed.pkl', 'rb') as f:
    sents = pickle.load(f)

In [4]:
def recover(tokens):
    sent = ''.join(tokens)
    sent = sent.replace('▁', ' ')
    return sent[1:]

def example_of_word (sents, words, idx = None, by_word = None, N = 15):
    if by_word is not None:
        word = by_word
        print(f'Target word: {word}')
        print(f'Word in new word list: {word in words}')
    elif idx is not None:
        word = words[idx]
        print(f'Target word: {word}')
    
    answer = []
    cowords = defaultdict(int)
    for sent in sents:
        comparable = list(map(lambda x: x.replace('▁', ''), sent))
        if word in comparable:
            answer.append(recover(sent))
            for w in comparable:
                cowords[w] += 1
    print(f'TOTAL SENTENCES: {len(answer)}')
    sorted_cowords = list(map(lambda x: x[0], sorted(cowords.items(), key = itemgetter(1), reverse = True)))
    new_cowords = {w : cowords[w] for w in sorted_cowords if w in words and w != word}
    TopN = dict(sorted(new_cowords.items(), key=itemgetter(1), reverse=True)[:N])
    print(f'TOP {N} NEAR NEW WORDS: {TopN}')
    return answer

In [5]:
example_of_word(sents, words, by_word = '1557')

Target word: 1557
Word in new word list: False
TOTAL SENTENCES: 2849
TOP 15 NEAR NEW WORDS: {'개좆슼': 376, '^1557^': 368, '슼갈': 189, '개좆슼갈': 123, '했누': 98, '느그즙': 58, '슼갈들': 52, '페이커': 49, '개좆즙': 49, '강딱': 47, '젠첩': 45, 'lck': 40, '는거': 37, '는건': 33, 'IG': 30}


['(깨알상식) 롤에서 넥서스가 깨지는 시간과 경기가 끝나는 시간은 다르다, 1557은 넥서스가 깨진 시간이고 실제 기록매체에선 경기시간을 게임이 끝난 16분 01초로 표현한다',
 '1557은 좀 심하지 ᄏᄏ -',
 '아 슼갈들 기인잘하니까 또 몰아가기하네 ᄏᄏ 그래 니네 선수가 최고야 역시 1557!!',
 '1557 9009 ^앞궁노존야스턴미포궁^',
 '1557 레전드 ᄏᄏ',
 '역시 1557 사이언스 ᄏᄏ',
 '팩트)1557때는 웃으면서 존나 깠지만 이제는 안된다 -',
 '롤드컵 못나간다니까 부들대노 1557좆즙갈새기들 ᄏᄏ',
 '1557한테 롤챔후장따이던 새끼들 존나웃기노ᄏᄏ',
 '-1557 단체정모-',
 'ᄂ쳐발리고 1557치니까 귀엽노',
 '대회에서 1557당하게 생긴 닉네임이네 -',
 '1557 두판해도 퍼블안나오노 ᄏᄏ',
 '1557도 드븐으로 당한거아니냐? 그냥 테디가 드븐에 좀 약한것같던데',
 '1557같은 국제전 중요 경기에서 나오는 기록이 의미 있는 거지 저런 정규시즌 경기같은 건 아무 의미 없음 ᄏᄏ',
 '1557한테 싹서열정리당하고 msi못나온게 lcgay',
 '1557 묻을려고 별 개지랄을 다떠네 추하다 씹새끼들아',
 'DRX가 슈퍼팀인이유 메갈들이 예쁜 연예인보듯 시기 질투글이 하루에도 20개씩 올라옴 ᄏᄏ 그저 1557 슥갈니스트',
 '난 슼ᄀ은 아닌데, 1557은 그냥 천재지변 수준의 일종의 사고였어. 실력차가 그만큼 심한건아니었다지?',
 '티원한테 1557 보냈으면 그 팬들이 그선수 매장시킨다.',
 '질문 좆같은거 사실인데? 사상검증:1557 느그즙 세체미 도인비',
 '1557 못하니까 15분 넘어가면 다들 병신되서 15분전까지 이기던거 다 갖다바쳤자너 ᄏᄏ',
 '역시 drx하고 비교도 안되는 역체갈. 그동안 맥천지새끼들이랑 비교해서 미안하다 너희가 최고다 씨발 1557 개졷슼새끼들아 !!!!!!!!@역시 drx하고 비교도 안되는 역체갈. 그동안 맥천지새끼들이랑 비교해서 미안하다 너희가 최

In [13]:
sent_checker = defaultdict(list)
for i in tqdm(range(len(sents))):
    for word in words:
        if word in list(map(lambda x: x.replace('▁', ''), sents[i])):
            sent_checker[i].append(word)

In [18]:
import json
with open('DC_LOL_sent_checker.json', 'w') as f:
    json.dump(sent_checker, f)

In [14]:
def find_topN (sent_checker = sent_checker, words = words, idx = None, by_word = None, N = 10, verbose = False):
    if by_word is not None:
        word = by_word
        if verbose:
            print(f'Target word: {word}')
            print(f'Word in new word list: {word in words}')
    elif idx is not None:
        word = words[idx]
        if verbose:
            print(f'Target word: {word}')
            
    cowords = defaultdict(int)
    for idx in list(sent_checker):
        sent = sent_checker[idx]
        if word in sent:
            for w in sent:
                cowords[w] += 1
                
    sorted_cowords = list(map(lambda x: x[0], sorted(cowords.items(), key = itemgetter(1), reverse = True)))
    new_cowords = {w : cowords[w] for w in sorted_cowords if w in words and w != word}
    TopN = dict(sorted(new_cowords.items(), key=itemgetter(1), reverse=True)[:N])
    if verbose:
        print(f'TOP {N} NEAR NEW WORDS: {TopN}')
    return TopN

In [17]:
result = {}
for i in tqdm(range(len(words))):
    topN = find_topN(idx = i)
    temp = {w: 0  for w in words}
    for w in topN:
        temp[w] = 1
    result[words[i]] = temp

In [19]:
import json
with open('DC_LOL_correlation.json', 'w') as f:
    json.dump(result, f)

In [20]:
df = pd.DataFrame(result)

In [21]:
mutual_pair = []
for i in tqdm(range(len(words))):
    for j in range(i, len(words)):
        word1 = words[i]
        word2 = words[j]
        if df.loc[word1, word2] and df.loc[word2, word1]:
            mutual_pair.append((word1, word2))

In [27]:
class DisjointSet:
    def __init__ (self, words):
        self.parent = {} # 각 성분의 parent를 포함하는 dict
        self.rank = {}   # 충돌하는 상황에서 발생하는 일들을 해결. 각 subtree의 성분의 수를 의미.
        for i in words:
            self.parent[i] = i
            self.rank[i] = 0
    
    def find (self, v):
        if self.parent[v] != v:
            self.parent[v] = self.find(self.parent[v])
        return self.parent[v]

    def union (self, root1, root2):
        if self.rank[root1] > self.rank[root2]:
            self.parent[root2] = root1
        else:
            self.parent[root1] = root2
            if self.rank[root1] == self.rank[root2]:
                self.rank[root2] += 1

def find_disjointset (words, info):
    disjointset = DisjointSet(words)

    for data in info:
        v,u = data
        root1 = disjointset.find(v)
        root2 = disjointset.find(u)
        if root1 != root2:
            disjointset.union(root1, root2)
            
    result = defaultdict(list)
    for k,v in disjointset.parent.items():
        result[v].append(k)
        
    return dict(result)

group = find_disjointset(words, mutual_pair)

In [26]:
def find_related_words(word):
    if word in df.index.tolist():
        return df.loc[df[word] == 1].index.tolist()
    else:
        print("NO SUCH WORD IN DataFrame.")
        return 0
    
find_related_words('빛태경')

['는거', '씨맥', 'LCK', '틀타', '잖어', '바미당', '대깨문', '자한당', '고있', '넣을게']